### Задача

К нам пришли наши коллеги из ML-отдела и рассказали, что планируют выкатывать новый алгоритм, рекомендующий нашим пользователям интересные посты. После обсуждений того, как он это делает, вы пришли к следующему пониманию:
- Алгоритм добавляет пользователям 1-2 просмотра
- Вероятность того, что он сработает, составляет 90%
- Если у пользователя меньше 50 просмотров, то алгоритм не сработает

Вы предполагаете, что увеличение числа просмотров приведёт и к увеличению лайков на пользователя. Встаёт вопрос: сможем ли мы обнаружить различия в среднем количестве лайков на пользователя? Чтобы ответить на этот вопрос, давайте проведём симуляцию Монте-Карло!

  Что мы будем делать:

- Распределения, из которых мы будем симулировать просмотры и пользовательские CTR, мы построим на основе периода АА-теста (даты смотрите в прошлом уроке). Выгрузите данные запросами, которые использовались в лекции, но уберите всё, связанное с exp_group. Данные нам понадобятся целиком, а на агрегацию эта переменная всё равно не повлияет.
- На эксперимент нам выделили неделю. Допустим, что за эту неделю в наш сервис зайдёт столько же пользователей, сколько зашло в период АА-теста. Мы планируем разбивать пользователей на две группы в соотношении 50/50. Посчитайте, сколько пользователей в таком случае придётся на одну группу.
- Эффект алгоритма на просмотры мы сымитируем следующим образом: group_B_views + ((1 + np.binomial(n=1, p=0.5, size=размер_выборки)) * np.binomial(n=1, p=0.9, size=размер_выборки) * (group_B_views >= 50)). Внимательно изучите эту строчку кода и подумайте, как она соотносится с описанием эффекта выше.
- Количество симуляций задайте не меньше 20000. Если хотите ещё больше уверенности в своих результатах — можете увеличить их число, но без фанатизма. 
- Лайки мы будем сравнивать t-тестом с поправкой Уэлча на неравные дисперсии (equal_var=False). Уровень значимости по классике поставим 0.05.

В ответе укажите получившееся значение мощности в процентах до первого знака после точки. Например, если у вас получится мощность, равная 0.919, то в ответе укажите 91.9.

In [1]:
# Выгружаем данные

import pandahouse as pandahouse
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

connection = {
    'host': 'https://clickhouse.lab.karpov.courses/',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator_20240620'
}

q = """
select views, count() as users
from (select user_id,
            sum(action = 'view') as views
from simulator_20240620.feed_actions 
where toDate(time) >= '2024-06-21' and toDate(time) <= '2024-06-27'
group by user_id
)
group by views
order by views
""" 

dist_view = pandahouse.read_clickhouse(q, connection=connection)
dist_view['p'] = dist_view['users']/dist_view['users'].sum()

qq = """
select 
   floor(ctr, 2) as ctr, count() as users
from ( select user_id,
            sum(action = 'like')/sum(action = 'view') as ctr
from simulator_20240620.feed_actions 
where toDate(time) >= '2024-06-21' and toDate(time) <= '2024-06-27'
group by user_id
)
group by ctr
"""

dist_ctr = pandahouse.read_clickhouse(qq, connection=connection)
dist_ctr['p'] = dist_ctr['users']/dist_ctr['users'].sum()

In [2]:
df = """
select  uniqExact(user_id)
FROM simulator_20240620.feed_actions 
WHERE toDate(time) >= '2024-06-21' and toDate(time) <= '2024-06-27'
"""

count_users = pandahouse.read_clickhouse(df, connection=connection)

q = """
select views, count() as users
from (select user_id,
            sum(action = 'view') as views
from simulator_20240620.feed_actions 
where toDate(time) >= '2024-06-21' and toDate(time) <= '2024-06-27'
group by user_id
)
group by views
order by views
""" 

dist_view = pandahouse.read_clickhouse(q, connection=connection)
dist_view['p'] = dist_view['users']/dist_view['users'].sum()
dist_view.head()

,views,users,p
0,1,23,0.000334
1,2,14,0.000203
2,3,6,0.000087
3,4,7,0.000102
4,5,21,0.000305


In [3]:
#сколько пользователей было в АА-тесте
df = """
select uniqExact(user_id)
from simulator_20240620.feed_actions
where toDate(time) >= '2024-06-21' and toDate(time) <= '2024-06-27'

"""

pandahouse.read_clickhouse(df, connection=connection)

,uniqExact(user_id)
0,68926


In [4]:
# 1 задание
rng = np.random.default_rng()
N = int(68926/2)

t_test = [] #список, куда мы будем складывать p-value
simulation = 20000

for _ in tqdm(range(simulation)):
    #симулируем выборки определённого размера несколько раз для просмотров
    group_A_views = rng.choice(dist_view['views'], size=N, replace=True, p=dist_view['p']).astype('int64')
    group_B_views = rng.choice(dist_view['views'], size=N, replace=True, p=dist_view['p']).astype('int64')
    group_B_views2 = (group_B_views + ((1 + rng.binomial(n=1, p=0.5, size=N)) * rng.binomial(n=1, p=0.9, size=N) * (group_B_views >= 50))).astype('int64')

    #симулируем выборки определённого размера несколько раз для ctr
    group_A_ctr = rng.choice(dist_ctr['ctr'], size=N, replace=True, p=dist_ctr['p'])
    group_B_ctr = rng.choice(dist_ctr['ctr'], size=N, replace=True, p=dist_ctr['p'])
    
    cliks_group_A = rng.binomial(group_A_views, group_A_ctr, size = N)
    cliks_group_B = rng.binomial(group_B_views2, group_B_ctr, size = N)
    t_test.append(stats.ttest_ind(cliks_group_A, cliks_group_B, equal_var=False)[1])

count_ = 0
for i in t_test:
    if i <= 0.05:
        count_ += 1
power1 = np.sum(count_)/simulation
print(power1*100)

100%|██████████| 20000/20000 [07:39<00:00, 43.56it/s]

34.27


В нашем случае статистическая мощность теста составляет 35%, это очень мало. Это говорит о том, что существует 65% вероятность того, что реальная разница между вариантами не будет обнаружена.

К нам снова пришли коллеги из ML-отдела с радостной новостью: они улучшили качество алгоритма! Теперь он срабатывает на пользователях с числом просмотров от 30 и выше.

Подкорректируйте вашу симуляцию и укажите новое значение получившейся мощности.

In [5]:
rng = np.random.default_rng()
N = int(68926/2)

t_test = []
simulation = 20000
for _ in tqdm(range(simulation)):
    #симулируем выборки определённого размера несколько раз для просмотров
    group_A_views = rng.choice(dist_view['views'], size=N, replace=True, p=dist_view['p']).astype('int64')
    group_B_views = rng.choice(dist_view['views'], size=N, replace=True, p=dist_view['p']).astype('int64')
    group_B_views2 = (group_B_views + ((1 + rng.binomial(n=1, p=0.5, size=N)) * rng.binomial(n=1, p=0.9, size=N) * (group_B_views >= 30))).astype('int64')

    #симулируем выборки определённого размера несколько раз для ctr
    group_A_ctr = rng.choice(dist_ctr['ctr'], size=N, replace=True, p=dist_ctr['p'])
    group_B_ctr = rng.choice(dist_ctr['ctr'], size=N, replace=True, p=dist_ctr['p'])
    
    cliks_group_A = rng.binomial(group_A_views, group_A_ctr, size = N)
    cliks_group_B = rng.binomial(group_B_views2, group_B_ctr, size = N)
    t_test.append(stats.ttest_ind(cliks_group_A, cliks_group_B, equal_var=False)[1])

count_ = 0
for i in t_test:
    if i <= 0.05:
        count_ += 1
power1 = np.sum(count_)/simulation
print(power1*100)

100%|██████████| 20000/20000 [07:32<00:00, 44.15it/s]

58.504999999999995


Теперь у нас мощность около 59% - то есть мы выиграли где-то 24% дополнительной мощности. Это по-прежнему мало.

Теперь нас пришло радовать начальство: нам утвердили длительность эксперимента длиной в 2 недели! Давайте теперь допустим, что в эти две недели к нам придёт столько же пользователей, сколько пришло суммарно за период АА-теста и АБ-теста (опять же, смотрите диапазон дат в прошлом уроке).

Подкорректируйте размер группы соответствующим образом. Сами распределения, на основе которых мы симулируем данные, менять не будем.

Какая мощность вышла теперь?

In [6]:
# 3 задание

import pandahouse as pandahouse
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

connection = {
    'host': 'https://clickhouse.lab.karpov.courses/',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator_20240620'
}

df = """
select  uniqExact(user_id)
FROM simulator_20240620.feed_actions 
WHERE toDate(time) >= '2024-06-08' and toDate(time) <= '2024-06-21'
"""

count_users = pandahouse.read_clickhouse(df, connection=connection)

q = """
select views, count() as users
from (select user_id,
            sum(action = 'view') as views
from simulator_20240620.feed_actions 
where toDate(time) >= '2024-06-21' and toDate(time) <= '2024-06-27'
group by user_id
)
group by views
order by views
""" 

dist_view = pandahouse.read_clickhouse(q, connection=connection)
dist_view['p'] = dist_view['users']/dist_view['users'].sum()

qq = """
select 
   floor(ctr, 2) as ctr, count() as users
from ( select user_id,
            sum(action = 'like')/sum(action = 'view') as ctr
from simulator_20240620.feed_actions 
where toDate(time) >= '2024-06-21' and toDate(time) <= '2024-06-27'
group by user_id
)
group by ctr
"""

dist_ctr = pandahouse.read_clickhouse(qq, connection=connection)
dist_ctr['p'] = dist_ctr['users']/dist_ctr['users'].sum()
count_users

,uniqExact(user_id)
0,81988


In [7]:

rng = np.random.default_rng()
N = int(81988/2)

t_test = []
simulation = 20000
for _ in tqdm(range(simulation)):
    #симулируем выборки определённого размера несколько раз для просмотров
    group_A_views = rng.choice(dist_view['views'], size=N, replace=True, p=dist_view['p']).astype('int64')
    group_B_views = rng.choice(dist_view['views'], size=N, replace=True, p=dist_view['p']).astype('int64')
    group_B_views2 = (group_B_views + ((1 + rng.binomial(n=1, p=0.5, size=N)) * rng.binomial(n=1, p=0.9, size=N) * (group_B_views >= 30))).astype('int64')

    #симулируем выборки определённого размера несколько раз для ctr
    group_A_ctr = rng.choice(dist_ctr['ctr'], size=N, replace=True, p=dist_ctr['p'])
    group_B_ctr = rng.choice(dist_ctr['ctr'], size=N, replace=True, p=dist_ctr['p'])
    
    cliks_group_A = rng.binomial(group_A_views, group_A_ctr, size = N)
    cliks_group_B = rng.binomial(group_B_views2, group_B_ctr, size = N)
    t_test.append(stats.ttest_ind(cliks_group_A, cliks_group_B, equal_var=False)[1])

count_ = 0
for i in t_test:
    if i <= 0.05:
        count_ += 1
power1 = np.sum(count_)/simulation
print(power1*100)

100%|██████████| 20000/20000 [09:01<00:00, 36.92it/s]

66.745


Всё это время мы анализировали наши выборки целиком — и тех пользователей, на которых алгоритм повлиял, и тех, кого он не мог затронуть (меньше 30 просмотров). А что, если мы будем отбирать только нужных пользователей и скармливать t-тесту именно их? Да, выборка будет меньше, но мы избавимся от мусора — а значит, и чувствительность наверняка будет выше.

Мы можем отсеять их примерно таким образом:
(здесь мы симулируем данные просмотров, CTR и лайков)

mask_A = group_A_views >= 30
mask_B = group_B_views >= 30

scipy.stats.ttest_ind(group_A_likes[mask_A], group_B_likes[mask_B], equal_var=False)

Какая мощность получилась сейчас? Дополнительно: какие выводы вы сделали для себя и о чём нам говорят эти результаты?

In [8]:
# 4 задание


rng = np.random.default_rng()
N = int(81988/2)

t_test = []
simulation = 20000
for _ in tqdm(range(simulation)):
    #симулируем выборки определённого размера несколько раз для просмотров
    group_A_views = rng.choice(dist_view['views'], size=N, replace=True, p=dist_view['p']).astype('int64')
    group_B_views = rng.choice(dist_view['views'], size=N, replace=True, p=dist_view['p']).astype('int64')
    group_B_views2 = (group_B_views + ((1 + rng.binomial(n=1, p=0.5, size=N)) * rng.binomial(n=1, p=0.9, size=N) * (group_B_views >= 30))).astype('int64')

    mask_A = group_A_views >= 30
    mask_B = group_B_views2 >= 30

    #симулируем выборки определённого размера несколько раз для ctr
    group_A_ctr = rng.choice(dist_ctr['ctr'], size=N, replace=True, p=dist_ctr['p'])
    group_B_ctr = rng.choice(dist_ctr['ctr'], size=N, replace=True, p=dist_ctr['p'])
    
    cliks_group_A = rng.binomial(group_A_views, group_A_ctr, size = N)
    cliks_group_B = rng.binomial(group_B_views2, group_B_ctr, size = N)
    t_test.append(stats.ttest_ind(cliks_group_A[mask_A], cliks_group_B[mask_B], equal_var=False)[1])
    
count_ = 0
for i in t_test:
    if i <= 0.05:
        count_ += 1
power1 = np.sum(count_)/simulation
print(power1*100)

100%|██████████| 20000/20000 [08:51<00:00, 37.64it/s]

76.765


Практически 80%, но все равно не хватает. В этом случае, можно сделать:
 - Отказаться от идеи эксперимента вообще. Если не хватает ресурсов для детекции такого изменения, то и смысла особо действовать нет.
 - Всё равно запустить эксперимент и надеяться на лучшее. Платой за это решение будет меньшая уверенность в полученных результатах.
 - Дорабатывать алгоритм, чтобы его эффект либо распространялся на большее число пользователей, либо чтобы он был больше.